# Analyzing IPTA DR2 Data

In this notebook we will use `enterprise` to analyze the IPTA DR2 for a stochastic GW background from DR2-lite where an initial cut of pulsars with > 10y baselines was made, then a FoM cut, and using the EPTA's J1713+0437 data.

In [1]:
% matplotlib inline
%config InlineBackend.figure_format = 'retina'
#%load_ext line_profiler

from __future__ import division

import numpy as np
import glob
import matplotlib.pyplot as plt
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise.signals import utils
from utils import *


import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

import json

Do not have mpi4py package.


### Notebook-specific choices

In [9]:
#The subset of pulsars to be analyzed
NG = False
EPTA = False
PPTA = False
PPTA4 = False
PPTA4_psrs = ['J0437-4715', 'J1713+0747', 'J1744-1134', 'J1909+1256']
specific_psrs = ['J0030+0451', 'J0218+4232', 'J0613-0200', 'J1012+5307', 'J1024-0719', 'J1713+0747', 'J1744-1134', \
                 'J1909-3744', 'J2145-0750', 'J0034-0534', 'J0437-4715', 'J0751+1807', 'J1022+1001', 'J1640+2224', \
                 'J1730-2304', 'J1857+0943', 'J1918-0642', 'J2317+1439']  # list of specific pulsars

#Path to directory containing the relevant par and tim files
datadir = '/home/tim/Documents/research/NANOGrav/ipta_busyweek_dec2017/IPTA_DR2_Lite_10yr/IPTA_DR2_Lite_10yr_partim'

#Path to directory containing the relevant .json noisefiles
noisedir = '/home/tim/Documents/research/NANOGrav/ipta_busyweek_dec2017/IPTA_DR2_Lite_10yr/IPTA_DR2_Lite_10yr_noisefiles'

#Path to directory where chains are stored
outdir = '/home/tim/Documents/research/NANOGrav/ipta_busyweek_dec2017/chains'

#Resume chains?
resume_chains = True

### Function to convert PAL2 noise parameters to enterprise parameter dict

In [10]:
def get_noise_from_pal2(noisefile):
    psrname = noisefile.split('/')[-1].split('_noise.txt')[0]
    fin = open(noisefile, 'r')
    lines = fin.readlines()
    params = {}
    for line in lines:
        ln = line.split()
        if 'efac' in line:
            par = 'efac'
            flag = ln[0].split('efac-')[-1]
        elif 'equad' in line:
            par = 'log10_equad'
            flag = ln[0].split('equad-')[-1]
        elif 'jitter_q' in line:
            par = 'log10_ecorr'
            flag = ln[0].split('jitter_q-')[-1]
        elif 'RN-Amplitude' in line:
            par = 'log10_A'
            flag = ''
        elif 'RN-spectral-index' in line:
            par = 'gamma'
            flag = ''
        else:
            break
        if flag:
            name = [psrname, flag, par]
        else:
            name = [psrname, par]
        pname = '_'.join(name)
        params.update({pname: float(ln[1])})
    return params

In [11]:
#Dictionary of all pulsars
psrdict_all = \
{'J0023+0923': [{'pta': ['NANOGrav']}],
 'J0030+0451': [{'pta': ['EPTA', 'NANOGrav']}],
 'J0034-0534': [{'pta': ['EPTA']}],
 'J0218+4232': [{'pta': ['EPTA']}],
 'J0340+4130': [{'pta': ['NANOGrav']}],
 'J0437-4715': [{'pta': ['PPTA']}],
 'J0610-2100': [{'pta': ['EPTA']}],
 'J0613-0200': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J0621+1002': [{'pta': ['EPTA']}],
 'J0645+5158': [{'pta': ['NANOGrav']}],
 'J0711-6830': [{'pta': ['PPTA']}],
 'J0751+1807': [{'pta': ['EPTA']}],
 'J0900-3144': [{'pta': ['EPTA']}],
 'J0931-1902': [{'pta': ['NANOGrav']}],
 'J1012+5307': [{'pta': ['EPTA', 'NANOGrav']}],
 'J1022+1001': [{'pta': ['EPTA']}],
 'J1024-0719': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1045-4509': [{'pta': ['PPTA']}],
 'J1455-3330': [{'pta': ['EPTA', 'NANOGrav']}],
 'J1600-3053': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1603-7202': [{'pta': ['PPTA']}],
 'J1614-2230': [{'pta': ['NANOGrav']}],
 'J1640+2224': [{'pta': ['EPTA', 'NANOGrav']}],
 'J1643-1224': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1713+0747': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1721-2457': [{'pta': ['EPTA']}],
 'J1730-2304': [{'pta': ['EPTA']}],
 'J1738+0333': [{'pta': ['EPTA', 'NANOGrav']}],
 'J1741+1351': [{'pta': ['NANOGrav']}],
 'J1744-1134': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1747-4036': [{'pta': ['NANOGrav']}],
 'J1751-2857': [{'pta': ['EPTA']}],
 'J1801-1417': [{'pta': ['EPTA']}],
 'J1802-2124': [{'pta': ['EPTA']}],
 'J1804-2717': [{'pta': ['EPTA']}],
 'J1824-2452A': [{'pta': ['PPTA']}],
 'J1832-0836': [{'pta': ['NANOGrav']}],
 'J1843-1113': [{'pta': ['EPTA']}],
 'J1853+1303': [{'pta': ['EPTA', 'NANOGrav']}],
 'J1857+0943': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1903+0327': [{'pta': ['NANOGrav']}],
 'J1909-3744': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1910+1256': [{'pta': ['EPTA', 'NANOGrav']}],
 'J1911+1347': [{'pta': ['EPTA']}],
 'J1911-1114': [{'pta': ['EPTA']}],
 'J1918-0642': [{'pta': ['EPTA', 'NANOGrav']}],
 'J1923+2515': [{'pta': ['NANOGrav']}],
 'J1939+2134': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J1944+0907': [{'pta': ['NANOGrav']}],
 'J1949+3106': [{'pta': ['NANOGrav']}],
 'J1955+2908': [{'pta': ['EPTA', 'NANOGrav']}],
 'J2010-1323': [{'pta': ['EPTA', 'NANOGrav']}],
 'J2017+0603': [{'pta': ['NANOGrav']}],
 'J2019+2425': [{'pta': ['EPTA']}],
 'J2033+1734': [{'pta': ['EPTA']}],
 'J2043+1711': [{'pta': ['NANOGrav']}],
 'J2124-3358': [{'pta': ['EPTA', 'PPTA']}],
 'J2129-5721': [{'pta': ['PPTA']}],
 'J2145-0750': [{'pta': ['EPTA', 'NANOGrav', 'PPTA']}],
 'J2214+3000': [{'pta': ['NANOGrav']}],
 'J2229+2643': [{'pta': ['EPTA']}],
 'J2302+4442': [{'pta': ['NANOGrav']}],
 'J2317+1439': [{'pta': ['EPTA', 'NANOGrav']}],
 'J2322+2057': [{'pta': ['EPTA']}]}

psrlist = []
if NG: psrlist.extend([psr for psr in psrdict_all.keys() if 'NANOGrav' in psrdict_all[psr][0].values()[0]])
if EPTA: psrlist.extend([psr for psr in psrdict_all.keys() if 'EPTA' in psrdict_all[psr][0].values()[0]])
if PPTA: psrlist.extend([psr for psr in psrdict_all.keys() if 'PPTA' in psrdict_all[psr][0].values()[0]])
if PPTA4: psrlist.extend([psr for psr in PPTA4_psrs])
if len(specific_psrs): psrlist.extend([psr for psr in specific_psrs])
psrlist = list(np.unique(psrlist))

In [14]:
psrlist

['J0030+0451',
 'J0034-0534',
 'J0218+4232',
 'J0437-4715',
 'J0613-0200',
 'J0751+1807',
 'J1012+5307',
 'J1022+1001',
 'J1024-0719',
 'J1640+2224',
 'J1713+0747',
 'J1730-2304',
 'J1744-1134',
 'J1857+0943',
 'J1909-3744',
 'J1918-0642',
 'J2145-0750',
 'J2317+1439']

### Get par, tim, and noise files
Here we collect the tim and par files as well as noise files made from the `PAL2` code. These are the same par, tim, and noise files used in the 9-year analysis papers. We use the convienience function above to convert from `PAL2` noise files to `enterprise` parameter dictionaries.

In [15]:
parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))

# filter
parfiles = [x for x in parfiles if x.split('/')[-1].split('.')[0] in psrlist]
timfiles = [x for x in timfiles if x.split('/')[-1].split('.')[0] in psrlist]

In [16]:
len(parfiles)

18

### Load into Pulsar class list

In [17]:
psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem='DE436')
    psrs.append(psr)

### Get parameter dict from noisefiles

In [20]:
noisefiles = [glob.glob(noisedir+'/*%s*.json'%psr) for psr in psrlist]
noisefiles = [nf for psr in noisefiles for nf in psr]

In [21]:
params = {}
for nf in noisefiles:
    with open(nf, 'r') as fin:
        params.update(json.load(fin))

### Set up model

When setting up the model for our upper limit run we fix all of the white noise (EFAC, EQUAD, and ECORR) parameters to the values obtained from the noise files. This is done by using `Constant` parameters. In this case we do not specify a default value for all instances of that parameter but instead will set them, based on their initialized pulsar and backend specific name, later via the `set_default_params` method of `PTA`. 

Speaking of white noise parameters here, we also use the `Selection` object.

Another feature to notice is that we do not use a uniform prior on the log of the red noise or GWB amplitude. Instead we use a `LinearExp` prior (short for linear-exponent prior), that is a prior of the form $p(x)\propto 10^x$. This is how we can still use the log of the parameter to sample but place a uniform prior on the parameter itself. We do this for both the red noise and GWB amplitude parameters.

Next, in order to save on computing time we do not include spatial correlations here. Instead we model the GWB as a common red process across all pulsars. In `enterprise` we can do this with a simple trick. We pre-initialize the parameters before passing them to the `Signal` model. In this way the *same* parameter instance is used for all pulsars. Lastly, we fixt the spectral index of the GWB to be 13/3 (4.33) using the `Constant` parameter.

## Setup model

We will add some addition model components that are not part of the base enterprise

### 1. Exponential decay function to model "void" in J1713+0747

In [22]:
@signal_base.function
def exp_decay(toas, freqs, log10_Amp=-7, t0=54000, log10_tau=1.7):
    t0 *= const.day
    tau = 10**log10_tau * const.day
    wf = - 10**log10_Amp * np.heaviside(toas-t0, 1) * np.exp(-(toas-t0)/tau)
    return wf * (1400/freqs)**2

### 2. Yearly DM sinusoid

In [23]:
@signal_base.function
def yearly_sinusoid(toas, freqs, log10_Amp=-7, phase=0):

    wf = 10**log10_Amp * np.sin(2*np.pi*const.fyr*toas+phase)
    return wf * (1400/freqs)**2

@signal_base.function
def yearly_sinusoid_basis(toas, freqs):
    
    F = np.zeros((len(toas), 2))
    F[:,0] = np.sin(2*np.pi*toas*const.fyr)
    F[:,1] = np.cos(2*np.pi*toas*const.fyr)
    
    Dm = (1400/freqs)**2

    return F * Dm[:, None], np.repeat(const.fyr, 2)

@signal_base.function
def yearly_sinusoid_prior(f):
    return np.ones(len(f)) * 1e20

### 3. DM EQUAD (EQUAD) term that scales like $\nu^{-4}$ (variance remember...)

In [24]:
# define DM EQUAD variance function
@signal_base.function
def dmequad_ndiag(freqs, log10_dmequad=-8):
    return np.ones_like(freqs) * (1400/freqs)**4 * 10**(2*log10_dmequad)

### 4. SVD timing model basis
This allows for more stability over standard scaling methods

In [25]:
# SVD timing model basis
@signal_base.function
def svd_tm_basis(Mmat):
    u, s, v = np.linalg.svd(Mmat, full_matrices=False)
    return u, np.ones_like(s)

@signal_base.function
def tm_prior(weights):
    return weights * 10**40

In [26]:
# find the maximum time span to set GW frequency sampling
tmin = np.min([p.toas.min() for p in psrs])
tmax = np.max([p.toas.max() for p in psrs])
Tspan = tmax - tmin

# define selection by observing backend
backends = selections.Selection(selections.by_backend)

# special selection for ECORR only use wideband NANOGrav data
NG_backends = selections.Selection(selections.nanograv_backends)

# white noise parameters
# since we are fixing these to values from the noise file we set
# them as constant parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

# red noise parameters
log10_A = parameter.LinearExp(-20, -11)
gamma = parameter.Uniform(0, 7)

# GW parameters (initialize with names here to use parameters in common across pulsars)
log10_A_gw = parameter.LinearExp(-18,-12)('log10_A_gw')
gamma_gw = parameter.Constant(4.33)('gamma_gw')

# DM turnover parameters
kappa = parameter.Uniform(0,7)
lf0 = parameter.Uniform(-9, -6.5)
log10_A_DM = parameter.Uniform(-20, -11)

# DM exponential parameters
t0 = parameter.Uniform(tmin/86400, tmax/86400)
log10_Amp = parameter.Uniform(-10, -2)
log10_tau = parameter.Uniform(np.log10(5), np.log10(500))

# DM sinusoid parameters
log10_Amp_s = parameter.Uniform(-10, -2)
phase = parameter.Uniform(0, 2*np.pi)

# white noise signals

# DM EQUAD
#dmvariance = dmequad_ndiag(log10_dmequad=equad)
#dmeq = white_signals.WhiteNoise(dmvariance)

# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=backends)
eq = white_signals.EquadNoise(log10_equad=equad, selection=backends)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=NG_backends)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)


# DM GP signals (use turnover model for more flexibility)
dm_basis = utils.createfourierdesignmatrix_dm(nmodes=30)
dm_prior = utils.turnover(log10_A=log10_A_DM, gamma=gamma, lf0=lf0, kappa=kappa)
dmgp = gp_signals.BasisGP(dm_prior, dm_basis, name='dm')


# DM exponential model
wf = exp_decay(log10_Amp=log10_Amp, t0=t0, log10_tau=log10_tau)
dmexp = deterministic_signals.Deterministic(wf, name='exp')

# DM sinusoid model
ys_prior = yearly_sinusoid_prior()
ys_basis = yearly_sinusoid_basis()
dmys = gp_signals.BasisGP(ys_prior, ys_basis, name='s1yr')

# gwb (no spatial correlations)
cpl = utils.powerlaw(log10_A=log10_A_gw, gamma=gamma_gw)
gw = gp_signals.FourierBasisGP(spectrum=cpl, components=30, Tspan=Tspan)

# for spatial correltions you can do...
#orf = utils.hd_orf()
#crn = gp_signals.FourierBasisCommonGP(cpl, orf, components=30, name='gw', Tspan=Tspan)

# to add solar system ephemeris modeling...
eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)

# timing model
basis = svd_tm_basis()
prior = tm_prior()
tm = gp_signals.BasisGP(prior, basis)
#tm = gp_signals.TimingModel()

# full model
#s = ef + eq + rn + tm + dmgp + dmys #+ dmeq
s = ef + eq + rn + tm + dmgp + dmys + eph + gw #+ dmeq

# intialize PTA, adding an exponential dip for the DM event in J1713+0747
models = []

for p in psrs:    
    if 'NANOGrav' in p.flags['pta']:
        s2 = s + ec 
        if p.name == 'J1713+0747':
            s3 = s2 + dmexp
            models.append(s3(p))
        else:
            models.append(s2(p))
    else:
        models.append(s(p))
    
pta = signal_base.PTA(models)

### Set white noise parameters

In [27]:
pta.set_default_params(params)

INFO: enterprise.signals.signal_base: Setting J0030+0451_430_ASP_efac to 1.14267230026
INFO: enterprise.signals.signal_base: Setting J0030+0451_L-wide_ASP_efac to 1.12477599181
INFO: enterprise.signals.signal_base: Setting J0030+0451_430_PUPPI_efac to 0.974529540585
INFO: enterprise.signals.signal_base: Setting J0030+0451_L-wide_PUPPI_efac to 1.14718012248
INFO: enterprise.signals.signal_base: Setting J0030+0451_430_ASP_log10_equad to -8.23082322696
INFO: enterprise.signals.signal_base: Setting J0030+0451_L-wide_PUPPI_log10_equad to -6.34303251049
INFO: enterprise.signals.signal_base: Setting J0030+0451_L-wide_ASP_log10_equad to -8.16192298684
INFO: enterprise.signals.signal_base: Setting J0030+0451_430_PUPPI_log10_equad to -5.95875161227
INFO: enterprise.signals.signal_base: Setting J0030+0451_430_ASP_log10_ecorr to -8.26646637907
INFO: enterprise.signals.signal_base: Setting J0030+0451_L-wide_ASP_log10_ecorr to -8.36191295728
INFO: enterprise.signals.signal_base: Setting J0030+0451_4

### Set initial parameters drawn from prior and evaluate likelihood to fill caches

Evaluating the likelihood is not necessary, the caches will be filled the first time it is called within the sampler if not called here.

In [28]:
xs = {par.name: par.sample() for par in pta.params}
print pta.get_lnlikelihood(xs);
print pta.get_lnprior(xs);

385646.699917
-54.363170815


In [33]:
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

### Set up sampler

In [34]:
# dimension of parameter space
#ndim = len(xs)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

# set up jump groups by red noise groups 

groups = get_parameter_groups(pta)

sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, groups=groups, 
                 outDir=outdir,resume=resume_chains)

### Sample!

In [ ]:
# sampler for N steps
N = int(1e6)
x0 = np.hstack(p.sample() for p in pta.params)
jp = JumpProposal(pta)
sampler.addProposalToCycle(jp.draw_from_prior, 15)
sampler.addProposalToCycle(jp.draw_from_ephem_prior, 15)
sampler.addProposalToCycle(jp.draw_from_gwb_log_uniform_prior, 10)
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

### Plot output

In [ ]:
chain = np.loadtxt(outdir + '/chain_1.txt')
#pars = sorted(xs.keys())
burn = int(0.25 * chain.shape[0])

In [ ]:
plt.hist(chain[burn:,-5], 50, normed=True, histtype='step', lw=2);
#plt.xlabel(pars[-1]);

### Upper limit value

We see that the upper limit agrees perfectly with the published value.

In [ ]:
upper = 10**np.percentile(chain[burn:, -5], q=95)
print(upper)